In [1]:
import pickle
# path = "data/Physics/0_uncopynode.pkl"
# with open(path, 'rb') as f:
#     data = pickle.load(f)
# print(data.edge_index.max())
# print(len(data.x))

In [41]:
# # print(data.edge_index.shape, data.train_mask.shape)
# # s = set()
# # for e in data.edge_index[1]:
# #     s.add(e.item())
# # print(len(s))
# # data.edge_index
# # data.train_mask

# # path = 'data/SBM/0_uncopynode.pkl'
# # with open(path, 'rb') as f:
# #     data = pickle.load(f)
# # print(data.y.min(),data.y.max())
# # print(data.edge_index[0].max())
# # print(data.edge_index[1].max())
# # print(len(data.x[0]))
# import torch
# x = torch.Tensor([0,0])
# print(x.to('cuda').grad)


None


In [3]:
import pickle
class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self

path_sbm_train = "data/SBM/SBM_CLUSTER_train.pkl"
path_sbm_val = "data/SBM/SBM_CLUSTER_val.pkl"
path_sbm_test = "data/SBM/SBM_CLUSTER_test.pkl"
with open(path_sbm_train, 'rb') as f:
    sbm_train = pickle.load(f)
with open(path_sbm_val, 'rb') as f:
    sbm_val = pickle.load(f)
with open(path_sbm_test, 'rb') as f:
    sbm_test = pickle.load(f)
# sbm_train

In [1]:
# # for tmp in sbm_train:
# #     print(len(tmp.node_feat))
# # sbm_train[0].node_feat
# # edge_index = torch.tensor([[],[]])
# # print(edge_index)
# # print(torch.cat((edge_index, torch.nonzero(sbm_train[0].W).T+100), 1))
# # x = torch.BoolTensor([0]*10)
# # x[1:3] = True
# # print(x)
# # print([1,2]*2)
# torch.nonzero(sbm_train[0].node_feat)

# torch.nonzero(sbm_train[0].W).T[:,torch.nonzero(sbm_train[0].W).T[0] == 20]
# for index in [  9,  13,  15,  19,  23,  24,  26,  31,  35,  41,  51,  52,  55,  58,
#           64,  65,  67,  72,  75,  76,  78,  80,  84,  86,  87,  89,  90,  92,
#           93,  94,  96,  99, 101, 102, 105, 111, 115]:
#     print(sbm_train[0].node_label[index])
# # s_train = 0
# # for tmp in sbm_train:
# #     s_train += len(tmp.node_feat)
# # s_val = 0
# # for tmp in sbm_val:
# #     s_val += len(tmp.node_feat)
# # s_test = 0
# # for tmp in sbm_test:
# #     s_test += len(tmp.node_feat)
# # print(s_train, s_val, s_test, s_train + s_val+s_test)


In [4]:
from torch_geometric.data import Data
K=1000
NBTRAIN = len(sbm_train)
NBVAL = len(sbm_val)
NBTEST = len(sbm_test)

import torch
for k in range(K):
    print(k)
    idx_base=0
    edge_index = torch.LongTensor([[],[]])
    x = torch.tensor([])
    y = torch.LongTensor([])
    for idx_graph in range(k*int(NBTRAIN/K),(k+1)*int(NBTRAIN/K)):
        sub_graph = sbm_train[idx_graph]
        edge_index = torch.cat(
            (edge_index, torch.nonzero(sub_graph.W).T+idx_base), 1)
        x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
        y = torch.cat((y, sub_graph.node_label), 0)
        idx_base += len(sub_graph.node_feat)
    # print(x.size(), edge_index, idx_base)
    idx_train = idx_base
    for idx_graph in range(k*int(NBVAL/K), (k+1)*int(NBVAL/K)):
        sub_graph = sbm_val[idx_graph]
        edge_index = torch.cat(
            (edge_index, torch.nonzero(sub_graph.W).T+idx_base), 1)
        x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
        y = torch.cat((y, sub_graph.node_label), 0)
        idx_base += len(sub_graph.node_feat)
    # print(x.size(), edge_index, idx_base)
    idx_val = idx_base
    for idx_graph in range(k*int(NBTEST/K), (k+1)*int(NBTEST/K)):
        sub_graph = sbm_test[idx_graph]
        edge_index = torch.cat(
            (edge_index, torch.nonzero(sub_graph.W).T+idx_base), 1)
        x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
        y = torch.cat((y, sub_graph.node_label), 0)
        idx_base += len(sub_graph.node_feat)
    # print(x.size(), edge_index, idx_base)
    idx_test = idx_base

    x = torch.zeros(x.shape[0], 7).scatter(1, torch.tensor(x,dtype=torch.int64), 1)[:, 1:]
    train_mask = torch.BoolTensor([0]*idx_base)
    train_mask[0:idx_train] = True
    val_mask = torch.BoolTensor([0]*idx_base)
    val_mask[idx_train:idx_val] = True
    test_mask = torch.BoolTensor([0]*idx_base)
    test_mask[idx_val:idx_test] = True
    labels = y[train_mask]
    weight = (labels.size(0) - torch.bincount(labels)).float() / labels.size(0)
    with open("data/SBM/{}_uncopynode.pkl".format(k), "wb") as f:
        pickle.dump(Data(edge_index=edge_index, x = x, y = y, train_mask = train_mask, val_mask = val_mask, test_mask = test_mask, weight=weight), f)
    # break


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


/tmp/ipykernel_25787/1710591562.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.zeros(x.shape[0], 7).scatter(1, torch.tensor(x,dtype=torch.int64), 1)[:, 1:]


33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299

In [31]:
# from torch_geometric.data import Data
# K=1000
# NBTRAIN = len(sbm_train)
# NBVAL = len(sbm_val)
# NBTEST = len(sbm_test)

# import torch
# for k in range(K):
#     print(k)
#     idx_base=0
#     edge_index = torch.LongTensor([[],[]])
#     x = torch.tensor([])
#     y = torch.LongTensor([])
#     for idx_graph in range(k*int(NBTRAIN/K),min((k+1)*int(NBTRAIN/K),NBTRAIN)):
#         sub_graph = sbm_train[idx_graph]
#         edge_index = torch.cat(
#             (edge_index, torch.nonzero(sub_graph.W).T+idx_base), 1)
#         x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
#         y = torch.cat((y, sub_graph.node_label), 0)
#         idx_base += len(sub_graph.node_feat)
#     # print(x.size(), edge_index, idx_base)
#     idx_train = idx_base
#     x = torch.zeros(x.shape[0], 7).scatter(1, torch.tensor(x,dtype=torch.int64), 1)[:, 1:]
#     with open("data/SBM/{}_train.pkl".format(k), "wb") as f:
#         pickle.dump(Data(edge_index=edge_index, x = x, y = y), f)
#     # break


In [1]:
from models import Gat, Sage#, Gcn
from torch import optim
import pickle
import torch.nn.functional as F
import utils
import torch.nn as nn
import torch
import torch_geometric.nn as gnn


# model = Gat(6,6).to('cuda:0')
datas = []
for i in range(1000):
    path = f'data/SBM/{i}_uncopynode.pkl'
    with open(path, 'rb') as f:
        datas.append(pickle.load(f).to('cuda:3'))


In [13]:
label = datas[0].y[datas[0].train_mask]
weight = (label.size(0) - torch.bincount(label)).float() / label.size(0)


tensor([True, True, True, True, True, True], device='cuda:3')

In [3]:
torch.cuda.manual_seed(0)
class Gcn(nn.Module):
    def __init__(self, nfeat, nclass):
        super(Gcn, self).__init__()
        hdim = 64
        self.lnin = nn.Linear(nfeat,hdim)
        self.convs = nn.ModuleList([gnn.GCNConv(hdim, hdim) for _ in range(4)])
        # self.lnout1 = nn.Linear(hdim, int(hdim/2), bias=True)
        # self.lnout2 = nn.Linear(int(hdim/2), int(hdim/4), bias=True)
        # self.lnout3 = nn.Linear(int(hdim/4), nclass, bias=True)
        self.lnout = nn.Linear(hdim, nclass)
        return

    def forward(self, x, edge_index):
        x = self.lnin(x)
        for conv in self.convs:
            x = x+F.relu(conv(x, edge_index))
        # x = F.relu(self.lnout1(x))
        # x = F.relu(self.lnout2(x))
        # x = self.lnout3(x)
        x = self.lnout(x)
        return x


class Sage(nn.Module):
    def __init__(self, nfeat, nclass):
        super(Sage, self).__init__()
        hdim = 162
        self.lnin = nn.Linear(nfeat, hdim)
        self.convs = nn.ModuleList(
            [gnn.SAGEConv(hdim, hdim) for _ in range(4)])
        self.lnout1 = nn.Linear(hdim, int(hdim/2), bias=True)
        self.lnout2 = nn.Linear(int(hdim/2), int(hdim/4), bias=True)
        self.lnout3 = nn.Linear(int(hdim/4), nclass, bias=True)
        return

    def forward(self, x, edge_index):
        x = self.lnin(x)
        for conv in self.convs:
            h_in = x
            x = F.relu(conv(x, edge_index))
            x = h_in + x
        x = F.relu(self.lnout1(x))
        x = F.relu(self.lnout2(x))
        x = self.lnout3(x)
        return x


class Gat(nn.Module):
    def __init__(self, nfeat, nclass):
        super(Gat, self).__init__()
        hdim = 162
        self.lnin = nn.Linear(nfeat, hdim)
        self.convs = nn.ModuleList(
            [gnn.GATConv(hdim, hdim) for _ in range(4)])
        self.lnout1 = nn.Linear(hdim, int(hdim/2), bias=True)
        self.lnout2 = nn.Linear(int(hdim/2), int(hdim/4), bias=True)
        self.lnout3 = nn.Linear(int(hdim/4), nclass, bias=True)
        return

    def forward(self, x, edge_index):
        x = self.lnin(x)
        for conv in self.convs:
            h_in = x
            x = F.elu(conv(x, edge_index))
            x = h_in + x
        x = F.relu(self.lnout1(x))
        x = F.relu(self.lnout2(x))
        x = self.lnout3(x)
        return x


class Sgc(nn.Module):
    def __init__(self, nfeat, nclass):
        super(Sgc, self).__init__()
        hdim = 162
        self.lnin = nn.Linear(nfeat, hdim)
        self.convs = nn.ModuleList(
            [gnn.SGConv(hdim, hdim, 1) for _ in range(4)])
        self.lnout1 = nn.Linear(hdim, int(hdim/2), bias=True)
        self.lnout2 = nn.Linear(int(hdim/2), int(hdim/4), bias=True)
        self.lnout3 = nn.Linear(int(hdim/4), nclass, bias=True)
        return

    def forward(self, x, edge_index):
        x = self.lnin(x)
        for conv in self.convs:
            h_in = x
            x = F.relu(conv(x, edge_index))
            x = h_in + x
        x = F.relu(self.lnout1(x))
        x = F.relu(self.lnout2(x))
        x = self.lnout3(x)
        return x


class Appnp(nn.Module):
    def __init__(self, nfeat, nclass):
        super(Appnp, self).__init__()
        hdim = 162
        self.lnin = nn.Linear(nfeat, hdim)
        self.convs = nn.ModuleList(
            [gnn.APPNP(K=1,alpha=0.1) for _ in range(4)])
        self.lnout1 = nn.Linear(hdim, int(hdim/2), bias=True)
        self.lnout2 = nn.Linear(int(hdim/2), int(hdim/4), bias=True)
        self.lnout3 = nn.Linear(int(hdim/4), nclass, bias=True)
        return

    def forward(self, x, edge_index):
        x = self.lnin(x)
        for conv in self.convs:
            h_in = x
            x = F.relu(conv(x, edge_index))
            x = h_in + x
        x = F.relu(self.lnout1(x))
        x = F.relu(self.lnout2(x))
        x = self.lnout3(x)
        return x


class Agnn(nn.Module):
    def __init__(self, nfeat, nclass):
        super(Agnn, self).__init__()
        hdim = 162
        self.lnin = nn.Linear(nfeat, hdim)
        self.convs = nn.ModuleList(
            [gnn.AGNNConv() for _ in range(4)])
        self.lnout1 = nn.Linear(hdim, int(hdim/2), bias=True)
        self.lnout2 = nn.Linear(int(hdim/2), int(hdim/4), bias=True)
        self.lnout3 = nn.Linear(int(hdim/4), nclass, bias=True)
        return

    def forward(self, x, edge_index):
        x = self.lnin(x)
        for conv in self.convs:
            h_in = x
            x = F.relu(conv(x, edge_index))
            x = h_in + x
        x = F.relu(self.lnout1(x))
        x = F.relu(self.lnout2(x))
        x = self.lnout3(x)
        return x


class Arma(nn.Module):
    def __init__(self, nfeat, nclass):
        super(Arma, self).__init__()
        hdim = 162
        self.lnin = nn.Linear(nfeat, hdim)
        self.convs = nn.ModuleList(
            [gnn.ARMAConv(hdim, hdim, 2) for _ in range(4)])
        self.lnout1 = nn.Linear(hdim, int(hdim/2), bias=True)
        self.lnout2 = nn.Linear(int(hdim/2), int(hdim/4), bias=True)
        self.lnout3 = nn.Linear(int(hdim/4), nclass, bias=True)
        return

    def forward(self, x, edge_index):
        x = self.lnin(x)
        for conv in self.convs:
            h_in = x
            x = F.relu(conv(x, edge_index))
            x = h_in + x
        x = F.relu(self.lnout1(x))
        x = F.relu(self.lnout2(x))
        x = self.lnout3(x)
        return x


class Gated(nn.Module):
    def __init__(self, nfeat, nclass):
        super(Gated, self).__init__()
        hdim = 70
        self.lnin = nn.Linear(nfeat, hdim)
        self.convs = nn.ModuleList(
            [gnn.GatedGraphConv(hdim, 1) for _ in range(4)])
        self.lnout1 = nn.Linear(hdim, int(hdim/2), bias=True)
        self.lnout2 = nn.Linear(int(hdim/2), int(hdim/4), bias=True)
        self.lnout3 = nn.Linear(int(hdim/4), nclass, bias=True)
        return

    def forward(self, x, edge_index):
        x = self.lnin(x)
        for conv in self.convs:
            h_in = x
            x = F.relu(conv(x, edge_index))
            x = h_in + x
        x = F.relu(self.lnout1(x))
        x = F.relu(self.lnout2(x))
        x = self.lnout3(x)
        return x


model = Gcn(6, 6).to('cuda:3')
model.train()
print(utils.num_params(model))

optimizer = optim.Adam(
    model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5)
for epoch in range(1000):
    losses = 0
    accus = 0
    test_accus = 0
    for iter, data in enumerate(datas):
        preds = model(data.x, data.edge_index)
        labels = data.y[data.train_mask]
        loss = F.cross_entropy(preds[data.train_mask], data.y[data.train_mask], weight=data.weight)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses += loss.item()
        accus += utils.accuracy(preds[data.train_mask],data.y[data.train_mask])
        test_accus += utils.accuracy(preds[data.test_mask],data.y[data.test_mask])
    print('Iter{}, loss:{},train_acc:{},test_acc:{}'.format(
        epoch+1, losses/(iter+1), accus/(iter+1),test_accus/(iter+1)))
    scheduler.step(losses/(iter+1))


17478
Iter1, loss:1.5836444767713547,train_acc:0.35456593555893157,test_acc:0.35730682588086243
Iter2, loss:1.434618411540985,train_acc:0.44584811438165356,test_acc:0.4494415812304131
Iter3, loss:1.3959058432579041,train_acc:0.46402776776391386,test_acc:0.46684484571630824
Iter4, loss:1.3741841263771057,train_acc:0.47536912476556054,test_acc:0.47901100156879867
Iter5, loss:1.3591267910003662,train_acc:0.48330062767404186,test_acc:0.4848536169076288
Iter6, loss:1.347815717935562,train_acc:0.4889950101968066,test_acc:0.4892791744227844
Iter7, loss:1.337858922123909,train_acc:0.4932774323673668,test_acc:0.4934152512373485
Iter8, loss:1.3287830805778504,train_acc:0.49736935000250043,test_acc:0.49798153311120685


KeyboardInterrupt: 